In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"shreeyash6199","key":"fa0ace82f613ab882fd3fb0b94916db0"}'}

In [0]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

example_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
fer2013.tar.gz: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!tar -xvf fer2013.tar.gz

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/


In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df = pd.read_csv('fer2013/fer2013.csv')

In [0]:
lst = []
lst.append([np.array(l.split(),dtype='int') for l in df.pixels[:33000]])
arr=np.array(lst)
X_train = arr.reshape((33000,48,48,1))

In [0]:
lst = []
lst.append([np.array(l.split(),dtype='int') for l in df.pixels[33000:]])
arr=np.array(lst)
X_test = arr.reshape((2887,48,48,1))

In [0]:
emotion = np.array(df.emotion,dtype='int')
emotion_dict = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}

In [10]:
from keras.utils import to_categorical
Y_train = to_categorical(emotion[:33000],num_classes=7)
Y_test = to_categorical(emotion[33000:],num_classes=7)

Using TensorFlow backend.


In [0]:
import keras
from keras.layers import Conv2D,Dense,MaxPool2D,concatenate,Input,BatchNormalization,Flatten
from keras.models import Model
from keras.optimizers import  Adam

def inception_module(previous):
    X1 = Conv2D(32,(1,1),padding='same',activation='relu')(previous)
    X2 = Conv2D(32,(3,3),padding='same',activation='relu')(X1)
    X3 = Conv2D(32,(5,5),padding='same',activation='relu')(X1)
    pool = MaxPool2D(padding='same',strides=1,pool_size=(2,2))(previous)
    X4 = Conv2D(32,(1,1),activation='relu',padding='same')(pool)
    incept = concatenate([X1,X2,X3,X4],axis=-1)
    return incept

inputs = Input(shape=(48,48,1))
X = Conv2D(32,(3,3),padding='same',activation='relu')(inputs)
X = MaxPool2D(padding='same',pool_size=(2,2),strides=1)(X)
X = BatchNormalization()(X)
"""
X = Conv2D(64,(3,3),padding='same',activation='relu')(X)
X = MaxPool2D(padding='same',pool_size=(2,2),strides=1)(X)
X = BatchNormalization()(X)
"""
X = inception_module(X)
#X = inception_module(X)
X = Flatten()(X)
X = Dense(1024,activation='relu')(X)
outputs = Dense(7,activation='softmax')(X)
inception = Model(inputs = inputs,outputs=outputs)



In [0]:
adam = Adam(0.008)
inception.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
inception.fit(X_train,Y_train,validation_split = 0.05,epochs=1)

Train on 31350 samples, validate on 1650 samples
Epoch 1/1


In [14]:
loss,acc = inception.evaluate(X_test,Y_test)
print('accuracy : ',acc)

2887/2887 [==============================] - 77s 27ms/step
accuracy :  0.24593003120519813


In [15]:
inception.save('inception_faces.h5')
files.download('inception_faces.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 44540, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil